In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.feature_selection import VarianceThreshold

In [2]:
pd.options.display.max_columns = None

In [3]:
df = pd.read_csv('/Users/jordonez/Desktop/Brainstation/CapstoneRepo/data/checkpoint3.csv')

In [4]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['date'] = pd.to_datetime(df['date'])

In [5]:
# we need to standardize by taking into account inflation rates for different years in the us
# https://www.usinflationcalculator.com/
# this how much $1 in 2024 would be in the stated year

new_price_ratios = {
    2012: 0.73,
    2013: 0.74,
    2014: 0.75,
    2015: 0.75,
    2016: 0.76,
    2017: 0.78,
    2018: 0.80,
    2019: 0.81,
    2020: 0.82,
    2021: 0.86,
    2022: 0.93,
    2023: 0.97,
    2024: 1.00
}

Objective: Adjust all of the prices in the data to match they would be in 2024 (accounting for inflation). 
We need to adjust the following prices:

1. median_sale_price
2. median_list_price
3. median_ppsf
4. median_list_ppsf

In [6]:
# we need to make a function given a month, year an inflation rate returns the adjusted price

def adjustPriceForInflation(oldPrice, year):
    new_price = round(oldPrice * new_price_ratios[year],2)
    return new_price

In [7]:
df_copy = df.copy(deep=True)

In [8]:
# Apply the inflation adjustment to the specified columns
df_copy['median_sale_price'] = df_copy.apply(lambda row: adjustPriceForInflation(row['median_sale_price'], row['year']), axis=1)
df_copy['median_list_price'] = df_copy.apply(lambda row: adjustPriceForInflation(row['median_list_price'], row['year']), axis=1)
df_copy['median_ppsf'] = df_copy.apply(lambda row: adjustPriceForInflation(row['median_ppsf'], row['year']), axis=1)
df_copy['median_list_ppsf'] = df_copy.apply(lambda row: adjustPriceForInflation(row['median_list_ppsf'], row['year']), axis=1)

In [9]:
# investigating other variables

In [10]:
df_copy['quarter'] = pd.to_datetime(df_copy['date']).dt.quarter

In [11]:
def get_season(month):
    if month in [12, 1, 2]:
        return 0
    elif month in [3, 4, 5]:
        return 1
    elif month in [6, 7, 8]:
        return 2
    else:
        return 3

df_copy['season'] = df_copy['month'].apply(get_season)

In [12]:
df_copy['price_diff'] = df_copy['median_list_price'] - df_copy['median_sale_price']

In [13]:
df_copy['ppsf_diff'] = df_copy['median_list_ppsf'] - df_copy['median_ppsf']

In [14]:
df_copy['sale_to_list_ratio'] = df_copy['median_sale_price'] / df_copy['median_list_price']

In [15]:
df_copy['price_growth'] = df_copy.groupby(['city', 'property_type'])['median_sale_price'].pct_change()

In [16]:
df_copy['owner_utility'] = (df_copy['median_list_price'] - df_copy['median_sale_price']) / df_copy['median_list_price']

In [17]:
df_copy['buyer_utility'] = (df_copy['median_sale_price'] - df_copy['median_list_price']) / df_copy['median_sale_price']

In [18]:
df_copy['pending_sales_ratio'] = df_copy['pending_sales'] / df_copy['new_listings']

In [19]:
df_copy['sales_success_rate'] = df_copy['homes_sold'] / df_copy['new_listings']

In [20]:
df_copy['inventory_turnover'] = df_copy['homes_sold'] / df_copy['inventory']

In [21]:
df_copy['pct_sold_above_list'] = df_copy['sold_above_list'] / df_copy['homes_sold']

In [22]:
df_copy['price_drop_rate'] = df_copy['price_drops'] / df_copy['new_listings']

In [23]:
df_copy['adjusted_months_supply'] = df_copy['months_of_supply'] / df_copy['homes_sold']

In [24]:
df_copy['political_stance_encoded'] = df_copy['senate_political_stance'].map({'democratic': 1, 'republican': 0, 'mixed': 2})

In [25]:
df_copy['supply_to_list_ratio'] = df_copy['inventory'] / df_copy['new_listings']

In [26]:
df_copy = pd.get_dummies(df_copy, columns=['property_type'], dtype='int')

In [27]:
df_copy['state_avg_sale_price'] = df_copy.groupby('state')['median_sale_price'].transform('mean')

In [28]:
df_copy['metro_region_inventory_change'] = df_copy.groupby('parent_metro_region')['inventory'].pct_change()

In [29]:
df_copy['dom_per_home'] = df_copy['median_dom'] / df_copy['homes_sold']

In [30]:
df_copy['inventory_to_pending_ratio'] = df_copy['inventory'] / df_copy['pending_sales']

In [31]:
df_copy['rolling_median_sale_price'] = df_copy['median_sale_price'].rolling(window=3).mean()

In [32]:
df_copy['rolling_median_list_price'] = df_copy['median_list_price'].rolling(window=3).mean()

In [33]:
df_copy['price_momentum'] = df_copy['median_sale_price'].diff() / df_copy['median_sale_price'].shift(1)

In [34]:
df_copy['supply_pressure'] = df_copy['new_listings'] / df_copy['homes_sold']

In [35]:
df_copy['demand_pressure'] = df_copy['pending_sales'] / df_copy['inventory']

In [36]:
df_copy['price_elasticity'] = df_copy['pending_sales'].pct_change() / df_copy['median_sale_price'].pct_change()

In [37]:
us_regions = {
    'New England': ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut'],
    'Mid-Atlantic': ['New York', 'New Jersey', 'Pennsylvania'],
    'East North Central': ['Ohio', 'Michigan', 'Indiana', 'Illinois', 'Wisconsin'],
    'West North Central': ['Minnesota', 'Iowa', 'Missouri', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas'],
    'South Atlantic': ['Delaware', 'Maryland', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida'],
    'East South Central': ['Kentucky', 'Tennessee', 'Mississippi', 'Alabama'],
    'West South Central': ['Arkansas', 'Louisiana', 'Texas', 'Oklahoma'],
    'Mountain': ['Montana', 'Idaho', 'Wyoming', 'Colorado', 'New Mexico', 'Arizona', 'Utah', 'Nevada'],
    'Pacific': ['Washington', 'Oregon', 'California', 'Alaska', 'Hawaii']
}


In [38]:
def binByUSRegion(state):
    for region, states in us_regions.items():
        if state in states:
            return region

In [39]:
df_copy['us_region'] = df_copy.apply(lambda row: binByUSRegion(row['state']), axis=1)

In [40]:
df_copy.us_region.unique()

array(['South Atlantic', 'Pacific', 'Mid-Atlantic', 'East South Central',
       'East North Central', 'West North Central', 'Mountain',
       'West South Central', 'New England'], dtype=object)

In [41]:
# make dummies of the us_region
df_copy = pd.get_dummies(df_copy, columns=['us_region'], dtype='int')

In [42]:
# drop columns with repeated data
df_copy = df_copy.drop(columns=['city', 'state','parent_metro_region','senate_political_stance'])

In [43]:
df_copy.isna().sum()

median_sale_price                              0
median_list_price                              0
median_ppsf                                    0
median_list_ppsf                               0
homes_sold                                     0
pending_sales                                  0
new_listings                                   0
inventory                                      0
months_of_supply                               0
median_dom                                     0
avg_sale_to_list                               0
sold_above_list                                0
price_drops                                    0
off_market_in_two_weeks                        0
year                                           0
month                                          0
date                                           0
quarter                                        0
season                                         0
price_diff                                     0
ppsf_diff           

In [44]:
df_copy = df_copy.dropna()

In [45]:
df_copy.isna().sum()

median_sale_price                          0
median_list_price                          0
median_ppsf                                0
median_list_ppsf                           0
homes_sold                                 0
pending_sales                              0
new_listings                               0
inventory                                  0
months_of_supply                           0
median_dom                                 0
avg_sale_to_list                           0
sold_above_list                            0
price_drops                                0
off_market_in_two_weeks                    0
year                                       0
month                                      0
date                                       0
quarter                                    0
season                                     0
price_diff                                 0
ppsf_diff                                  0
sale_to_list_ratio                         0
price_grow

In [46]:
df_copy.describe()

/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,months_of_supply,median_dom,avg_sale_to_list,sold_above_list,price_drops,off_market_in_two_weeks,year,month,date,quarter,season,price_diff,ppsf_diff,sale_to_list_ratio,price_growth,owner_utility,buyer_utility,pending_sales_ratio,sales_success_rate,inventory_turnover,pct_sold_above_list,price_drop_rate,adjusted_months_supply,political_stance_encoded,supply_to_list_ratio,property_type_All Residential,property_type_Condo/Co-op,property_type_Multi-Family (2-4 Unit),property_type_Single Family Residential,property_type_Townhouse,state_avg_sale_price,metro_region_inventory_change,dom_per_home,inventory_to_pending_ratio,rolling_median_sale_price,rolling_median_list_price,price_momentum,supply_pressure,demand_pressure,price_elasticity,us_region_East North Central,us_region_East South Central,us_region_Mid-Atlantic,us_region_Mountain,us_region_New England,us_region_Pacific,us_region_South Atlantic,us_region_West North Central,us_region_West South Central
count,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5427216,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06,5.427216e+06
mean,2.616219e+05,2.874394e+05,1.609351e+02,1.757599e+02,1.470899e+01,1.400407e+01,1.966011e+01,4.714562e+01,4.168112e+00,9.913312e+01,9.737626e-01,2.326873e-01,2.654781e-01,2.782676e-01,2.018096e+03,6.510994e+00,2018-07-22 02:43:56.790574336,2.499805e+00,1.517455e+00,2.581752e+04,1.482488e+01,9.863783e-01,1.983274e+00,1.362170e-02,-2.128982e+00,1.517052e+00,9.479263e-01,4.971490e-01,8.587737e-02,7.821524e-02,2.075089e+00,1.089802e+00,3.321353e+00,3.869271e-01,9.731140e-02,5.613762e-02,3.794616e-01,8.016228e-02,2.605019e+05,5.362082e+00,5.381633e+01,5.842027e+00,2.614856e+05,2.873419e+05,4.737850e+00,3.362540e+00,1.049135e+00,NaN,1.668616e-01,4.836365e-02,1.836098e-01,6.176500e-02,7.216444e-02,1.363500e-01,1.843527e-01,7.248283e-02,7.404994e-02
min,7.300000e-01,2.175000e+02,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,5.000000e-01,0.000000e+00,1.090869e-04,0.000000e+00,2.012000e+03,1.000000e+00,2012-01-01 00:00:00,1.000000e+00,0.000000e+00,-7.979200e+07,-1.119051e+07,3.271188e-06,-9.999977e-01,-2.700149e+03,-3.056984e+05,4.084130e-03,3.076923e-03,8.598452e-04,0.000000e+00,6.348213e-08,0.000000e+00,0.000000e+00,9.429197e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.307783e+05,-9.999520e-01,2.396645e-03,1.097303e-02,4.175000e+03,1.581333e+04,-9.999991e-01,1.115242e-02,1.169591e-03,-inf,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.064000e+05,1.301172e+05,6.770000e+01,7.813000e+01,1.000000e+00,2.000000e+00,3.000000e+00,5.000000e+00,1.700000e+00,2.900000e+01,9.547502e-01,0.000000e+00,1.818182e-01,0.000000e+00,2.015000e+03,4.000000e+00,2015-09-01 00:00:00,2.000000e+00,1.000000e+00,-1.912500e+04,-7.350000e+00,6.850439e-01,-2.180451e-01,-1.045545e-01,-4.597605e-01,4.863823e-01,4.000000e-01,1.666667e-01,0.000000e+00,1.100324e-02,2.000000e-01,1.000000e+00,1.305875e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.602289e+05,-7.088608e-01,3.187500e+00,1.283178e+00,1.418250e+05,1.668375e+05,-5.003125e-01,9.166667e-01,1.538462e-01,-1.214073e+00,0.000000e+00,0.000000e+00,0.000000e

In [47]:
elasticity_is_inf = (df_copy['price_elasticity'] == float('inf')) | (df_copy['price_elasticity'] == float('-inf'))
df_copy.loc[elasticity_is_inf]

,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,months_of_supply,median_dom,avg_sale_to_list,sold_above_list,price_drops,off_market_in_two_weeks,year,month,date,quarter,season,price_diff,ppsf_diff,sale_to_list_ratio,price_growth,owner_utility,buyer_utility,pending_sales_ratio,sales_success_rate,inventory_turnover,pct_sold_above_list,price_drop_rate,adjusted_months_supply,political_stance_encoded,supply_to_list_ratio,property_type_All Residential,property_type_Condo/Co-op,property_type_Multi-Family (2-4 Unit),property_type_Single Family Residential,property_type_Townhouse,state_avg_sale_price,metro_region_inventory_change,dom_per_home,inventory_to_pending_ratio,rolling_median_sale_price,rolling_median_list_price,price_momentum,supply_pressure,demand_pressure,price_elasticity,us_region_East North Central,us_region_East South Central,us_region_Mid-Atlantic,us_region_Mountain,us_region_New England,us_region_Pacific,us_region_South Atlantic,us_region_West North Central,us_region_West South Central
2684,51100.0,68948.5,51.17,47.28,3.0,2.000000,10.000000,46.000000,15.3,57.0,0.940853,0.000000,0.022727,0.000000,2012,1,2012-01-01,1,0,17848.5,-3.89,0.741133,-0.199543,0.258867,-0.349286,0.200000,0.300000,0.065217,0.000000,0.002273,5.100000,2,4.600000,1,0,0,0,0,140394.243047,14.333333,19.000000,23.000000,48423.333333,95711.516667,0.0,3.333333,0.043478,-inf,1,0,0,0,0,0,0,0,0
8193,81030.0,100302.0,39.92,80.14,2.0,4.617643,2.000000,31.000000,15.5,170.0,0.978564,0.000000,0.174256,0.122225,2012,1,2012-01-01,1,0,19272.0,40.22,0.807860,-0.603571,0.192140,-0.237838,2.308821,1.000000,0.064516,0.000000,0.087128,7.750000,1,15.500000,1,0,0,0,0,286615.516753,0.068966,85.000000,6.713382,62780.000000,101421.333333,0.0,1.000000,0.148956,inf,0,0,1,0,0,0,0,0,0
14768,112237.5,109427.0,72.93,76.53,16.0,6.000000,12.915145,71.689593,5.3,92.0,0.981102,0.000000,0.188112,0.000000,2012,1,2012-01-01,1,0,-2810.5,3.60,1.025684,0.282319,-0.025684,0.025041,0.464571,1.238856,0.223184,0.000000,0.014565,0.331250,0,5.550816,0,0,0,1,0,149892.839862,-0.158911,5.750000,11.948265,155514.333333,177317.000000,0.0,0.807197,0.083694,inf,0,0,0,0,0,0,0,0,1
19928,109500.0,47012.0,47.78,33.93,2.0,4.617643,2.000000,21.000000,10.5,201.0,0.936888,0.000000,0.174256,0.122225,2012,1,2012-01-01,1,0,-62488.0,-13.85,2.329193,1.307692,-1.329193,0.570667,2.308821,1.000000,0.095238,0.000000,0.087128,5.250000,1,10.500000,1,0,0,0,0,286615.516753,6.000000,100.500000,4.547775,135475.833333,124781.333333,0.0,1.000000,0.219888,-inf,0,0,1,0,0,0,0,0,0
25177,65700.0,726350.0,77.48,199.77,1.0,4.617643,1.000000,56.000000,56.0,165.0,0.900901,0.000000,0.174256,0.122225,2012,2,2012-02-01,1,0,660650.0,122.29,0.090452,-0.217391,0.909548,-10.055556,4.617643,1.000000,0.017857,0.000000,0.174256,56.000000,1,56.000000,1,0,0,0,0,286615.516753,55.000000,165.000000,12.127400,71783.333333,310688.000000,0.0,1.000000,0.082458,inf,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5482396,345000.0,380990.0,271.72,219.37,2.0,10.490476,1.000000,1.000000,0.5,45.0,0.961003,0.000000,0.282157,0.147578,2024,6,2024-06-01,2,2,35990.0,-52.35,0.905536,0.169492,0.094464,-0.104319,10.490476,2.000000,2.000000,0.000000,0.282157,0.250000,0,1.000000,0,1,0,0,0,204226.249476,-0.981818,22.500000,0.095325,930000.000000,663478.333333,0.0,0.500000,10.490476,-inf,0,0,0,0,0,0,1,0,0
5482446,255000.0,282450.0,240.79,223.87,5.0,3.000000,6.000000,8.000000,1.6,34.0,0.968216,0.000000,0.250000,0.333333,2024,6,2024-06-01,2,2,27450.0,-16.92,0.902815,-0.046729,0.097185,-0.107647,0.500000,0.833333,0.625000,0.000000,0.041667,0.320000,1,1.333333,1,0,0,0,0,542114.131124,0.142857,6.800000,2.666667,263716.666667,324450.000000,0.0,1.200000,0.375000,-inf,0,0,0,0,0,1,0,0,0
5482495,785000.0,1350000.0,705.09,8

In [48]:
# let's drop these rows with -inf or inf
df_copy = df_copy.drop(df_copy.loc[elasticity_is_inf].index)

In [49]:
# finally round off number of homes sold since can't have decimal homes
df['homes_sold'] = round(df['homes_sold'])

In [50]:
df_copy.describe()

,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,months_of_supply,median_dom,avg_sale_to_list,sold_above_list,price_drops,off_market_in_two_weeks,year,month,date,quarter,season,price_diff,ppsf_diff,sale_to_list_ratio,price_growth,owner_utility,buyer_utility,pending_sales_ratio,sales_success_rate,inventory_turnover,pct_sold_above_list,price_drop_rate,adjusted_months_supply,political_stance_encoded,supply_to_list_ratio,property_type_All Residential,property_type_Condo/Co-op,property_type_Multi-Family (2-4 Unit),property_type_Single Family Residential,property_type_Townhouse,state_avg_sale_price,metro_region_inventory_change,dom_per_home,inventory_to_pending_ratio,rolling_median_sale_price,rolling_median_list_price,price_momentum,supply_pressure,demand_pressure,price_elasticity,us_region_East North Central,us_region_East South Central,us_region_Mid-Atlantic,us_region_Mountain,us_region_New England,us_region_Pacific,us_region_South Atlantic,us_region_West North Central,us_region_West South Central
count,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5420909,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06,5.420909e+06
mean,2.616948e+05,2.874912e+05,1.609777e+02,1.757154e+02,1.470798e+01,1.400296e+01,1.965868e+01,4.714426e+01,4.168222e+00,9.913457e+01,9.737643e-01,2.326896e-01,2.654708e-01,2.782586e-01,2.018096e+03,6.511013e+00,2018-07-22 00:11:29.362761472,2.499811e+00,1.517453e+00,2.579636e+04,1.473775e+01,9.864332e-01,1.985286e+00,1.356679e-02,-2.131023e+00,1.516825e+00,9.479957e-01,4.971668e-01,8.586697e-02,7.820949e-02,2.074889e+00,1.089814e+00,3.321446e+00,3.869334e-01,9.729309e-02,5.612601e-02,3.794869e-01,8.016054e-02,2.605115e+05,5.361722e+00,5.381016e+01,5.842618e+00,2.615269e+05,2.873754e+05,4.743363e+00,3.361803e+00,1.048910e+00,2.121198e+00,1.668589e-01,4.836384e-02,1.836061e-01,6.177156e-02,7.216963e-02,1.363683e-01,1.843399e-01,7.247309e-02,7.404865e-02
min,7.300000e-01,2.175000e+02,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,5.000000e-01,0.000000e+00,1.090869e-04,0.000000e+00,2.012000e+03,1.000000e+00,2012-01-01 00:00:00,1.000000e+00,0.000000e+00,-7.979200e+07,-1.119051e+07,3.271188e-06,-9.999977e-01,-2.700149e+03,-3.056984e+05,4.084130e-03,3.076923e-03,8.598452e-04,0.000000e+00,6.348213e-08,0.000000e+00,0.000000e+00,9.429197e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.307783e+05,-9.999520e-01,2.396645e-03,1.097303e-02,4.175000e+03,1.581333e+04,-9.999991e-01,1.115242e-02,1.169591e-03,-9.990000e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.064000e+05,1.301172e+05,6.770000e+01,7.813000e+01,1.000000e+00,2.000000e+00,3.000000e+00,5.000000e+00,1.700000e+00,2.900000e+01,9.547530e-01,0.000000e+00,1.818182e-01,0.000000e+00,2.015000e+03,4.000000e+00,2015-09-01 00:00:00,2.000000e+00,1.000000e+00,-1.912500e+04,-7.350000e+00,6.850685e-01,-2.180396e-01,-1.045929e-01,-4.597080e-01,4.863823e-01,4.000000e-01,1.666667e-01,0.000000e+00,1.100113e-02,2.000000e-01,1.000000e+00,1.306122e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.602289e+05,-7.088451e-01,3.187500e+00,1.284162e+00,1.418400e+05,1.668515e+05,-5.009737e-01,9.166667e-01,1.538462e-01,-1.209686e+00,0.000000e+00,0.00

In [51]:
df_copy.sample(10)

,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,months_of_supply,median_dom,avg_sale_to_list,sold_above_list,price_drops,off_market_in_two_weeks,year,month,date,quarter,season,price_diff,ppsf_diff,sale_to_list_ratio,price_growth,owner_utility,buyer_utility,pending_sales_ratio,sales_success_rate,inventory_turnover,pct_sold_above_list,price_drop_rate,adjusted_months_supply,political_stance_encoded,supply_to_list_ratio,property_type_All Residential,property_type_Condo/Co-op,property_type_Multi-Family (2-4 Unit),property_type_Single Family Residential,property_type_Townhouse,state_avg_sale_price,metro_region_inventory_change,dom_per_home,inventory_to_pending_ratio,rolling_median_sale_price,rolling_median_list_price,price_momentum,supply_pressure,demand_pressure,price_elasticity,us_region_East North Central,us_region_East South Central,us_region_Mid-Atlantic,us_region_Mountain,us_region_New England,us_region_Pacific,us_region_South Atlantic,us_region_West North Central,us_region_West South Central
1196242,129525.0,155025.00,78.89,83.76,11.0,3.000000,16.000000,73.000000,6.600000,146.0,0.948417,0.181818,0.218731,0.000000,2015,3,2015-03-01,1,1,25500.00,4.87,0.835510,-0.088654,0.164490,-0.196873,0.187500,0.687500,0.150685,0.016529,0.013671,0.600000,2,4.562500,1,0,0,0,0,270850.334042,5.636364,13.272727,24.333333,96362.500000,109075.000000,0.074339,1.454545,0.041096,-5.380753,0,0,0,0,0,0,1,0,0
1842076,2432000.0,208499.57,536.04,114.62,1.0,20.989890,32.397088,11.000000,11.000000,202.0,0.805031,0.000000,0.090909,0.258844,2016,9,2016-09-01,3,3,-2223500.43,-421.42,11.664293,-0.873016,-10.664293,0.914268,0.647894,0.030867,0.090909,0.000000,0.002806,11.000000,2,0.339537,0,0,0,1,0,270850.334042,-0.450000,202.000000,0.524062,974168.000000,250982.783333,8.078014,32.397088,1.908172,0.433640,0,0,0,0,0,0,1,0,0
975939,120000.0,123675.00,67.25,74.26,19.0,6.000000,18.000000,82.000000,4.300000,113.0,0.971249,0.105263,0.046034,0.000000,2014,9,2014-09-01,3,3,3675.00,7.01,0.970285,0.038961,0.029715,-0.030625,0.333333,1.055556,0.231707,0.005540,0.002557,0.226316,0,4.555556,1,0,0,0,0,161783.637946,12.666667,5.947368,13.666667,211000.000000,187225.000000,1.500000,0.947368,0.073171,3.333333,0,1,0,0,0,0,0,0,0
30715,405150.0,583927.00,222.73,355.06,1.0,2.180929,1.000000,11.000000,11.000000,158.0,0.928094,0.000000,0.217871,0.160358,2012,2,2012-02-01,1,0,178777.00,132.33,0.693837,-0.339286,0.306163,-0.441261,2.180929,1.000000,0.090909,0.000000,0.217871,11.000000,1,11.000000,0,0,0,0,1,330640.499192,-0.312500,158.000000,5.043722,471555.666667,468854.666667,2.967119,1.000000,0.198266,0.030489,0,0,1,0,0,0,0,0,0
1034432,279375.0,269812.50,122.80,129.51,38.0,30.000000,34.000000,254.000000,6.700000,80.0,0.967450,0.078947,0.177165,0.000000,2014,11,2014-11-01,4,3,-9562.50,6.71,1.035441,0.138710,-0.035441,0.034228,0.882353,1.117647,0.149606,0.002078,0.005211,0.176316,2,7.470588,0,0,0,1,0,356310.532386,-0.198738,2.105263,8.466667,180375.000000,215537.500000,1.551370,0.894737,0.118110,0.966887,0,0,0,1,0,0,0,0,0
2057617,55302.0,93522.00,52.79,65.70,2.0,5.039216,1.000000,4.000000,2.000000,4.0,0.990818,0.000000,0.245805,0.152818,2017,3,2017-03-01,1,1,38220.00,12.91,0.591326,-0.185057,0.408674,-0.691114,5.039216,2.000000,0.500000,0.000000,0.245805,1.000000,2,4.000000,0,1,0,0,0,140394.243047,-0.882353,2.000000,0.793774,88114.000000,175878.300000,-0.580473,0.500000,1.259804,1.667787,1,0,0,0,0,0,0,0,0
3546408,422300.0,403645.00,130.18,137.78,1.0,1.000000,1.000000,6.000000,6.000000,28.0,1.030000,1.000000,0.333333,0.000000,2020,5,2020-05-01,2,1,-18655.00,7.60,1.046216,3.224774,-0.046216,0.044175,1.000000,1.000000,0.166667,1.000000,0.333333,6.000000,1,6.000000,0,0,0,0,1,207468.544210,-0.435299,28.000000,6.000000,358066.666667,388721.000000,0.337662,1.000000,0.166667,-2.591346,0,0,0,0,0,0,0,1,0
4915380,243470.0,302882.50,171.77,190.55,9.0,8.000000,26.000000,69.000000,7.700000,73.0,0.961224,0.222222,0.246377,0.250000,2023,

In [53]:
df_copy.to_csv('checkpoint4.csv')